In [1]:
import pandas as pd
import numpy as np
import plotly.express as px
pd.options.display.float_format = '{:.2f}'.format

In [2]:
# Read in New York Times online csv file with daily COVID data
# Convert FIPS number to integer, convert date to datetime, save file to local csv

us_covid_df = pd.read_csv('https://raw.githubusercontent.com/nytimes/covid-19-data/master/us-counties.csv')
us_covid_df['fips'] = us_covid_df['fips'].fillna(0).astype(np.int64)
us_covid_df['date'] = pd.to_datetime(us_covid_df['date'], yearfirst=True)
us_covid_df.to_csv('us_covid.csv', index=False)

In [3]:
# Read in national population data provided by state, county, FIPS

us_pops_df = pd.read_csv('US_Populations.csv', engine='python', thousands=',')
us_pops_df

,FIPStxt,State,State2,Area_Name,Region,population
0,2000,Alaska,AK,Alaska,State,731545
1,2013,Alaska,AK,Aleutians East Borough,County,3337
2,2016,Alaska,AK,Aleutians West Census Area,County,5634
3,2020,Alaska,AK,Anchorage,County,288000
4,2050,Alaska,AK,Bethel Census Area,County,18386
...,...,...,...,...,...,...
3146,22119,Louisiana,LA,Webster,County,38340
3147,22121,Louisiana,LA,West Baton Rouge,County,26465
3148,22123,Louisiana,LA,West Carroll,County,10830
3149,22125,Louisiana,LA,West Feliciana,County,15568


In [4]:
# Merge New York Times data with population data

us_data_df = pd.merge(us_covid_df, us_pops_df, left_on=['county', 'state'], right_on=['Area_Name', 'State'])
us_data_df

,date,county,state,fips,cases,deaths,FIPStxt,State,State2,Area_Name,Region,population
0,2020-01-21,Snohomish,Washington,53061,1,0.00,53061,Washington,WA,Snohomish,County,822083
1,2020-01-22,Snohomish,Washington,53061,1,0.00,53061,Washington,WA,Snohomish,County,822083
2,2020-01-23,Snohomish,Washington,53061,1,0.00,53061,Washington,WA,Snohomish,County,822083
3,2020-01-24,Snohomish,Washington,53061,1,0.00,53061,Washington,WA,Snohomish,County,822083
4,2020-01-25,Snohomish,Washington,53061,1,0.00,53061,Washington,WA,Snohomish,County,822083
...,...,...,...,...,...,...,...,...,...,...,...,...
1487953,2021-07-22,Kalawao,Hawaii,15005,1,0.00,15005,Hawaii,HI,Kalawao,County,86
1487954,2021-07-23,Kalawao,Hawaii,15005,1,0.00,15005,Hawaii,HI,Kalawao,County,86
1487955,2021-07-24,Kalawao,Hawaii,15005,1,0.00,15005,Hawaii,HI,Kalawao,County,86
1487956,2021-07-25,Kalawao,Hawaii,15005,1,0.00,15005,Hawaii,HI,Kalawao,County,86


In [5]:
# Separate out data from the state level

states = us_data_df[(us_data_df['Region'] == 'State')]
states

,date,county,state,fips,cases,deaths,FIPStxt,State,State2,Area_Name,Region,population
39175,2020-03-07,District of Columbia,District of Columbia,11001,1,0.00,11000,District of Columbia,DC,District of Columbia,State,705749
39177,2020-03-08,District of Columbia,District of Columbia,11001,1,0.00,11000,District of Columbia,DC,District of Columbia,State,705749
39179,2020-03-09,District of Columbia,District of Columbia,11001,4,0.00,11000,District of Columbia,DC,District of Columbia,State,705749
39181,2020-03-10,District of Columbia,District of Columbia,11001,4,0.00,11000,District of Columbia,DC,District of Columbia,State,705749
39183,2020-03-11,District of Columbia,District of Columbia,11001,10,0.00,11000,District of Columbia,DC,District of Columbia,State,705749
...,...,...,...,...,...,...,...,...,...,...,...,...
910148,2021-07-22,Arkansas,Arkansas,5001,2425,39.00,5000,Arkansas,AR,Arkansas,State,3017804
910150,2021-07-23,Arkansas,Arkansas,5001,2432,39.00,5000,Arkansas,AR,Arkansas,State,3017804
910152,2021-07-24,Arkansas,Arkansas,5001,2444,39.00,5000,Arkansas,AR,Arkansas,State,3017804
910154,2021-07-25,Arkansas,Arkansas,5001,2463,39.00,5000,Arkansas,AR,Arkansas,State,3017804


In [6]:
# Separate out data specific to county level, drop redundant columns
#  Create infection and death rates, updating for each date in the dataframe

counties = us_data_df.drop(['fips', 'State','Area_Name'], axis=1)
counties['current_crate'] = counties['cases']*100000/counties['population']
counties['current_drate'] = counties['deaths']*100000/counties['population']
counties = counties[(counties['Region'] == 'County')]
counties

,date,county,state,cases,deaths,FIPStxt,State2,Region,population,current_crate,current_drate
0,2020-01-21,Snohomish,Washington,1,0.00,53061,WA,County,822083,0.12,0.00
1,2020-01-22,Snohomish,Washington,1,0.00,53061,WA,County,822083,0.12,0.00
2,2020-01-23,Snohomish,Washington,1,0.00,53061,WA,County,822083,0.12,0.00
3,2020-01-24,Snohomish,Washington,1,0.00,53061,WA,County,822083,0.12,0.00
4,2020-01-25,Snohomish,Washington,1,0.00,53061,WA,County,822083,0.12,0.00
...,...,...,...,...,...,...,...,...,...,...,...
1487953,2021-07-22,Kalawao,Hawaii,1,0.00,15005,HI,County,86,1162.79,0.00
1487954,2021-07-23,Kalawao,Hawaii,1,0.00,15005,HI,County,86,1162.79,0.00
1487955,2021-07-24,Kalawao,Hawaii,1,0.00,15005,HI,County,86,1162.79,0.00
1487956,2021-07-25,Kalawao,Hawaii,1,0.00,15005,HI,County,86,1162.79,0.00


In [7]:
# Isolate data from most current day.

counties_us = counties[(counties['date'] == '2021-07-26') & (counties['Region'] == 'County')]
counties_us

,date,county,state,cases,deaths,FIPStxt,State2,Region,population,current_crate,current_drate
552,2021-07-26,Snohomish,Washington,42432,616.00,53061,WA,County,822083,5161.52,74.93
1102,2021-07-26,Cook,Illinois,562216,11109.00,17031,IL,County,5150233,10916.32,215.70
1651,2021-07-26,Orange,California,280149,5142.00,6059,CA,County,3175692,8821.67,161.92
2199,2021-07-26,Maricopa,Arizona,576443,10415.00,4013,AZ,County,4485414,12851.50,232.20
2747,2021-07-26,Los Angeles,California,1286456,24643.00,6037,CA,County,10039107,12814.45,245.47
...,...,...,...,...,...,...,...,...,...,...,...
1486933,2021-07-26,Wheeler,Oregon,36,1.00,41069,OR,County,1332,2702.70,75.08
1487220,2021-07-26,King,Texas,11,0.00,48269,TX,County,272,4044.12,0.00
1487476,2021-07-26,Esmeralda,Nevada,41,0.00,32009,NV,County,873,4696.45,0.00
1487728,2021-07-26,Loving,Texas,3,0.00,48301,TX,County,169,1775.15,0.00


In [8]:
# Create up to date rates so rankings can be performed using the latest data.

counties_us['drate'] = counties_us['deaths']/(counties_us['population']/100000)
counties_us['crate'] = counties_us['cases']/(counties_us['population']/100000)
counties_us = counties_us[(counties_us['Region'] == 'County')]
counties_us

<ipython-input-8-608f6ade14d5>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  counties_us['drate'] = counties_us['deaths']/(counties_us['population']/100000)
<ipython-input-8-608f6ade14d5>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  counties_us['crate'] = counties_us['cases']/(counties_us['population']/100000)


,date,county,state,cases,deaths,FIPStxt,State2,Region,population,current_crate,current_drate,drate,crate
552,2021-07-26,Snohomish,Washington,42432,616.00,53061,WA,County,822083,5161.52,74.93,74.93,5161.52
1102,2021-07-26,Cook,Illinois,562216,11109.00,17031,IL,County,5150233,10916.32,215.70,215.70,10916.32
1651,2021-07-26,Orange,California,280149,5142.00,6059,CA,County,3175692,8821.67,161.92,161.92,8821.67
2199,2021-07-26,Maricopa,Arizona,576443,10415.00,4013,AZ,County,4485414,12851.50,232.20,232.20,12851.50
2747,2021-07-26,Los Angeles,California,1286456,24643.00,6037,CA,County,10039107,12814.45,245.47,245.47,12814.45
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1486933,2021-07-26,Wheeler,Oregon,36,1.00,41069,OR,County,1332,2702.70,75.08,75.08,2702.70
1487220,2021-07-26,King,Texas,11,0.00,48269,TX,County,272,4044.12,0.00,0.00,4044.12
1487476,2021-07-26,Esmeralda,Nevada,41,0.00,32009,NV,County,873,4696.45,0.00,0.00,4696.45
1487728,2021-07-26,Loving,Texas,3,0.00,48301,TX,County,169,1775.15,0.00,0.00,1775.15


In [9]:
# Create ranking of infection rates by county for each state

counties_us.sort_values(by=['state', 'crate', 'county'], inplace=True, ascending=(True, False, True))
counties_us['state_case']=counties_us.groupby([counties_us.State2]).cumcount()+1
counties_us

,date,county,state,cases,deaths,FIPStxt,State2,Region,population,current_crate,current_drate,drate,crate,state_case
951863,2021-07-26,Hale,Alabama,2320,78.00,1065,AL,County,14651,15835.10,532.39,532.39,15835.10,1
1036483,2021-07-26,Clarke,Alabama,3649,62.00,1025,AL,County,23622,15447.46,262.47,262.47,15447.46,2
744081,2021-07-26,Lowndes,Alabama,1446,54.00,1085,AL,County,9726,14867.37,555.21,555.21,14867.37,3
659490,2021-07-26,Etowah,Alabama,14676,368.00,1055,AL,County,102268,14350.53,359.84,359.84,14350.53,4
553086,2021-07-26,Franklin,Alabama,4450,81.00,1059,AL,County,31362,14189.15,258.27,258.27,14189.15,5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
908701,2021-07-26,Sublette,Wyoming,842,7.00,56035,WY,County,9831,8564.74,71.20,71.20,8564.74,19
908215,2021-07-26,Converse,Wyoming,1116,19.00,56009,WY,County,13822,8074.08,137.46,137.46,8074.08,20
1130280,2021-07-26,Lincoln,Wyoming,1491,14.00,56023,WY,County,19830,7518.91,70.60,70.60,7518.91,21
1258886,2021-07-26,Niobrara,Wyoming,165,3.00,56027,WY,County,2356,7003.40,127.33,127.33,7003.40,22


In [10]:
# Create ranking of death rates by county for each state

counties_us.sort_values(by=['state', 'drate', 'county'], inplace=True, ascending=(True, False, True))
counties_us['state_death']=counties_us.groupby([counties_us.State2]).cumcount()+1
counties_us

,date,county,state,cases,deaths,FIPStxt,State2,Region,population,current_crate,current_drate,drate,crate,state_case,state_death
744081,2021-07-26,Lowndes,Alabama,1446,54.00,1085,AL,County,9726,14867.37,555.21,555.21,14867.37,3,1
951863,2021-07-26,Hale,Alabama,2320,78.00,1065,AL,County,14651,15835.10,532.39,532.39,15835.10,1,2
315673,2021-07-26,Walker,Alabama,7619,286.00,1127,AL,County,63521,11994.46,450.24,450.24,11994.46,30,3
657534,2021-07-26,Clay,Alabama,1627,59.00,1027,AL,County,13235,12293.16,445.79,445.79,12293.16,24,4
802638,2021-07-26,Greene,Alabama,980,36.00,1063,AL,County,8111,12082.36,443.84,443.84,12082.36,28,5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1130280,2021-07-26,Lincoln,Wyoming,1491,14.00,56023,WY,County,19830,7518.91,70.60,70.60,7518.91,21,19
800689,2021-07-26,Hot Springs,Wyoming,385,3.00,56017,WY,County,4413,8724.22,67.98,67.98,8724.22,18,20
1101960,2021-07-26,Uinta,Wyoming,2382,13.00,56041,WY,County,20226,11776.92,64.27,64.27,11776.92,5,21
314188,2021-07-26,Teton,Wyoming,3881,11.00,56039,WY,County,23464,16540.23,46.88,46.88,16540.23,1,22


In [11]:
# Merge data from today with historical data to provide current rates + rates from previous points in time.

usa = pd.merge(counties, counties_us[['state', 'county','drate', 'crate', 'state_death', 'state_case']], on=['state', 'county'])
usa

,date,county,state,cases,deaths,FIPStxt,State2,Region,population,current_crate,current_drate,drate,crate,state_death,state_case
0,2020-01-21,Snohomish,Washington,1,0.00,53061,WA,County,822083,0.12,0.00,74.93,5161.52,20,24
1,2020-01-22,Snohomish,Washington,1,0.00,53061,WA,County,822083,0.12,0.00,74.93,5161.52,20,24
2,2020-01-23,Snohomish,Washington,1,0.00,53061,WA,County,822083,0.12,0.00,74.93,5161.52,20,24
3,2020-01-24,Snohomish,Washington,1,0.00,53061,WA,County,822083,0.12,0.00,74.93,5161.52,20,24
4,2020-01-25,Snohomish,Washington,1,0.00,53061,WA,County,822083,0.12,0.00,74.93,5161.52,20,24
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1484488,2021-07-22,Kalawao,Hawaii,1,0.00,15005,HI,County,86,1162.79,0.00,0.00,1162.79,5,4
1484489,2021-07-23,Kalawao,Hawaii,1,0.00,15005,HI,County,86,1162.79,0.00,0.00,1162.79,5,4
1484490,2021-07-24,Kalawao,Hawaii,1,0.00,15005,HI,County,86,1162.79,0.00,0.00,1162.79,5,4
1484491,2021-07-25,Kalawao,Hawaii,1,0.00,15005,HI,County,86,1162.79,0.00,0.00,1162.79,5,4


In [12]:
# Break out daily case data from the New York Times csv that is cumulative in nature

usa.sort_values(by=['state', 'state_case', 'date'], inplace=True)
usa['case_day'] = usa.groupby(['state','county']).cases.diff().fillna(usa.cases)
usa

,date,county,state,cases,deaths,FIPStxt,State2,Region,population,current_crate,current_drate,drate,crate,state_death,state_case,case_day
947915,2020-03-30,Hale,Alabama,1,0.00,1065,AL,County,14651,6.83,0.00,532.39,15835.10,2,1,1.00
947916,2020-03-31,Hale,Alabama,1,0.00,1065,AL,County,14651,6.83,0.00,532.39,15835.10,2,1,0.00
947917,2020-04-01,Hale,Alabama,1,0.00,1065,AL,County,14651,6.83,0.00,532.39,15835.10,2,1,0.00
947918,2020-04-02,Hale,Alabama,1,0.00,1065,AL,County,14651,6.83,0.00,532.39,15835.10,2,1,0.00
947919,2020-04-03,Hale,Alabama,3,0.00,1065,AL,County,14651,20.48,0.00,532.39,15835.10,2,1,2.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1149324,2021-07-22,Crook,Wyoming,469,13.00,56011,WY,County,7584,6184.07,171.41,171.41,6197.26,6,23,0.00
1149325,2021-07-23,Crook,Wyoming,470,13.00,56011,WY,County,7584,6197.26,171.41,171.41,6197.26,6,23,1.00
1149326,2021-07-24,Crook,Wyoming,470,13.00,56011,WY,County,7584,6197.26,171.41,171.41,6197.26,6,23,0.00
1149327,2021-07-25,Crook,Wyoming,470,13.00,56011,WY,County,7584,6197.26,171.41,171.41,6197.26,6,23,0.00


In [13]:
# Break out daily death data from the New York Times csv that is cumulative in nature

usa.sort_values(by=['state', 'state_death', 'date'], inplace=True)
usa['death_day'] = usa.groupby(['state','county']).deaths.diff().fillna(usa.deaths)
usa

,date,county,state,cases,deaths,FIPStxt,State2,Region,population,current_crate,current_drate,drate,crate,state_death,state_case,case_day,death_day
741101,2020-03-26,Lowndes,Alabama,1,0.00,1085,AL,County,9726,10.28,0.00,555.21,14867.37,1,3,1.00,0.00
741102,2020-03-27,Lowndes,Alabama,1,0.00,1085,AL,County,9726,10.28,0.00,555.21,14867.37,1,3,0.00,0.00
741103,2020-03-28,Lowndes,Alabama,1,0.00,1085,AL,County,9726,10.28,0.00,555.21,14867.37,1,3,0.00,0.00
741104,2020-03-29,Lowndes,Alabama,1,0.00,1085,AL,County,9726,10.28,0.00,555.21,14867.37,1,3,0.00,0.00
741105,2020-03-30,Lowndes,Alabama,1,0.00,1085,AL,County,9726,10.28,0.00,555.21,14867.37,1,3,0.00,0.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
738656,2021-07-22,Albany,Wyoming,4712,18.00,56001,WY,County,38880,12119.34,46.30,46.30,12168.21,23,3,1.00,0.00
738657,2021-07-23,Albany,Wyoming,4716,18.00,56001,WY,County,38880,12129.63,46.30,46.30,12168.21,23,3,4.00,0.00
738658,2021-07-24,Albany,Wyoming,4716,18.00,56001,WY,County,38880,12129.63,46.30,46.30,12168.21,23,3,0.00,0.00
738659,2021-07-25,Albany,Wyoming,4716,18.00,56001,WY,County,38880,12129.63,46.30,46.30,12168.21,23,3,0.00,0.00


In [14]:
# Export updated national data to csv for use in Tableau

usa.to_csv('daily_covid_us.csv', index=False)

In [15]:
# Filter California data to show only numbers for today

counties_cal = usa[(usa['date'] == '2021-07-26') & (usa['state'] == 'California')]
counties_cal

,date,county,state,cases,deaths,FIPStxt,State2,Region,population,current_crate,current_drate,drate,crate,state_death,state_case,case_day,death_day
377021,2021-07-26,Imperial,California,29037,742.00,6025,CA,County,181215,16023.51,409.46,409.46,16023.51,1,2,41.00,1.00
2747,2021-07-26,Los Angeles,California,1286456,24643.00,6037,CA,County,10039107,12814.45,245.47,245.47,12814.45,2,5,1927.00,15.00
187389,2021-07-26,San Bernardino,California,304521,5237.00,6071,CA,County,2180085,13968.31,240.22,240.22,13968.31,3,4,446.00,1.00
661398,2021-07-26,Inyo,California,1446,38.00,6027,CA,County,18039,8015.97,210.65,210.65,8015.97,4,26,0.00,0.00
83629,2021-07-26,Stanislaus,California,65304,1085.00,6099,CA,County,550660,11859.22,197.04,197.04,11859.22,5,9,285.00,0.00
66495,2021-07-26,San Joaquin,California,76034,1446.00,6077,CA,County,762148,9976.28,189.73,189.73,9976.28,6,17,0.00,0.00
38667,2021-07-26,Riverside,California,305589,4658.00,6065,CA,County,2470546,12369.29,188.54,188.54,12369.29,7,7,394.00,0.00
103746,2021-07-26,Tulare,California,50221,854.00,6107,CA,County,466195,10772.53,183.19,183.19,10772.53,8,11,117.00,0.00
37653,2021-07-26,Fresno,California,104019,1742.00,6019,CA,County,999101,10411.26,174.36,174.36,10411.26,9,14,382.00,6.00
599185,2021-07-26,Merced,California,32961,484.00,6047,CA,County,277680,11870.14,174.30,174.30,11870.14,10,8,0.00,0.00


In [16]:
# Show top 10 California counties based on largest current infection rate

counties_cal.sort_values(by=['crate'], inplace=True, ascending=False)
top10_ca_case_rates = counties_cal.head(10)
top10_ca_case_rates

<ipython-input-16-94608615179b>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  counties_cal.sort_values(by=['crate'], inplace=True, ascending=False)


,date,county,state,cases,deaths,FIPStxt,State2,Region,population,current_crate,current_drate,drate,crate,state_death,state_case,case_day,death_day
1405978,2021-07-26,Lassen,California,5868,28.00,6035,CA,County,30573,19193.41,91.58,91.58,19193.41,35,1,0.00,0.00
377021,2021-07-26,Imperial,California,29037,742.00,6025,CA,County,181215,16023.51,409.46,409.46,16023.51,1,2,41.00,1.00
802580,2021-07-26,Kings,California,23489,249.00,6031,CA,County,152940,15358.31,162.81,162.81,15358.31,11,3,31.00,1.00
187389,2021-07-26,San Bernardino,California,304521,5237.00,6071,CA,County,2180085,13968.31,240.22,240.22,13968.31,3,4,446.00,1.00
2747,2021-07-26,Los Angeles,California,1286456,24643.00,6037,CA,County,10039107,12814.45,245.47,245.47,12814.45,2,5,1927.00,15.00
235737,2021-07-26,Kern,California,112623,1419.00,6029,CA,County,900202,12510.86,157.63,157.63,12510.86,14,6,305.00,2.00
38667,2021-07-26,Riverside,California,305589,4658.00,6065,CA,County,2470546,12369.29,188.54,188.54,12369.29,7,7,394.00,0.00
599185,2021-07-26,Merced,California,32961,484.00,6047,CA,County,277680,11870.14,174.30,174.30,11870.14,10,8,0.00,0.00
83629,2021-07-26,Stanislaus,California,65304,1085.00,6099,CA,County,550660,11859.22,197.04,197.04,11859.22,5,9,285.00,0.00
802093,2021-07-26,Colusa,California,2339,18.00,6011,CA,County,21547,10855.34,83.54,83.54,10855.34,36,10,0.00,0.00


In [17]:
# Show top 10 California counties based on largest current death rate

counties_cal.sort_values(by=['drate'], inplace=True, ascending=False)
top10_ca_death_rates = counties_cal.head(10)
top10_ca_death_rates

<ipython-input-17-018d0b919d54>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  counties_cal.sort_values(by=['drate'], inplace=True, ascending=False)


,date,county,state,cases,deaths,FIPStxt,State2,Region,population,current_crate,current_drate,drate,crate,state_death,state_case,case_day,death_day
377021,2021-07-26,Imperial,California,29037,742.00,6025,CA,County,181215,16023.51,409.46,409.46,16023.51,1,2,41.00,1.00
2747,2021-07-26,Los Angeles,California,1286456,24643.00,6037,CA,County,10039107,12814.45,245.47,245.47,12814.45,2,5,1927.00,15.00
187389,2021-07-26,San Bernardino,California,304521,5237.00,6071,CA,County,2180085,13968.31,240.22,240.22,13968.31,3,4,446.00,1.00
661398,2021-07-26,Inyo,California,1446,38.00,6027,CA,County,18039,8015.97,210.65,210.65,8015.97,4,26,0.00,0.00
83629,2021-07-26,Stanislaus,California,65304,1085.00,6099,CA,County,550660,11859.22,197.04,197.04,11859.22,5,9,285.00,0.00
66495,2021-07-26,San Joaquin,California,76034,1446.00,6077,CA,County,762148,9976.28,189.73,189.73,9976.28,6,17,0.00,0.00
38667,2021-07-26,Riverside,California,305589,4658.00,6065,CA,County,2470546,12369.29,188.54,188.54,12369.29,7,7,394.00,0.00
103746,2021-07-26,Tulare,California,50221,854.00,6107,CA,County,466195,10772.53,183.19,183.19,10772.53,8,11,117.00,0.00
37653,2021-07-26,Fresno,California,104019,1742.00,6019,CA,County,999101,10411.26,174.36,174.36,10411.26,9,14,382.00,6.00
599185,2021-07-26,Merced,California,32961,484.00,6047,CA,County,277680,11870.14,174.30,174.30,11870.14,10,8,0.00,0.00


In [18]:
# Create dataframe with historical data for Top 10 counties based on current case rate

top10c_df = top10_ca_case_rates[['county', 'state']]
top10_ca_case_df = pd.merge(usa, top10c_df, on=['county', 'state'])
top10_ca_case_df['date'] = pd.to_datetime(top10_ca_case_df['date']).dt.strftime('%m/%d/%y')
top10_ca_case_df = top10_ca_case_df[['date', 'county','cases', 'deaths', 'current_crate', 'current_drate', 'state_death', 'state_case', 'case_day', 'death_day']]
top10_ca_case_df

,date,county,cases,deaths,current_crate,current_drate,state_death,state_case,case_day,death_day
0,03/20/20,Imperial,4,0.00,2.21,0.00,1,2,4.00,0.00
1,03/21/20,Imperial,4,0.00,2.21,0.00,1,2,0.00,0.00
2,03/22/20,Imperial,4,0.00,2.21,0.00,1,2,0.00,0.00
3,03/23/20,Imperial,4,0.00,2.21,0.00,1,2,0.00,0.00
4,03/24/20,Imperial,9,0.00,4.97,0.00,1,2,5.00,0.00
...,...,...,...,...,...,...,...,...,...,...
4937,07/22/21,Colusa,2322,18.00,10776.44,83.54,36,10,0.00,0.00
4938,07/23/21,Colusa,2339,18.00,10855.34,83.54,36,10,17.00,0.00
4939,07/24/21,Colusa,2339,18.00,10855.34,83.54,36,10,0.00,0.00
4940,07/25/21,Colusa,2339,18.00,10855.34,83.54,36,10,0.00,0.00


In [19]:
# Create dataframe with historical data for Top 10 counties based on current death rate

top10d_df = top10_ca_death_rates[['county', 'state']]
top10_ca_death_df = pd.merge(usa, top10d_df, on=['county', 'state'])
top10_ca_death_df['date'] = pd.to_datetime(top10_ca_death_df['date']).dt.strftime('%m/%d/%y')
top10_ca_death_df = top10_ca_death_df[['date', 'county','cases', 'deaths', 'current_crate', 'current_drate', 'state_death', 'state_case', 'case_day', 'death_day']]
top10_ca_death_df

,date,county,cases,deaths,current_crate,current_drate,state_death,state_case,case_day,death_day
0,03/20/20,Imperial,4,0.00,2.21,0.00,1,2,4.00,0.00
1,03/21/20,Imperial,4,0.00,2.21,0.00,1,2,0.00,0.00
2,03/22/20,Imperial,4,0.00,2.21,0.00,1,2,0.00,0.00
3,03/23/20,Imperial,4,0.00,2.21,0.00,1,2,0.00,0.00
4,03/24/20,Imperial,9,0.00,4.97,0.00,1,2,5.00,0.00
...,...,...,...,...,...,...,...,...,...,...
5038,07/22/21,Merced,32814,484.00,11817.20,174.30,10,8,0.00,0.00
5039,07/23/21,Merced,32961,484.00,11870.14,174.30,10,8,147.00,0.00
5040,07/24/21,Merced,32961,484.00,11870.14,174.30,10,8,0.00,0.00
5041,07/25/21,Merced,32961,484.00,11870.14,174.30,10,8,0.00,0.00


In [20]:
# Export California Top 10 case data to csv

top10_ca_case_df.to_csv('top10cases.csv', float_format='%.2f', index=False)

In [21]:
# Export California Top 10 death data to csv

top10_ca_death_df.to_csv('top10deaths.csv', float_format='%.2f', index=False)